In [4]:

#from __future__ import division
import numpy as np
import warnings

In [5]:
import jax.numpy as jnp

In [6]:
# see if we got a compile time flag
# include '.compile_time_use_pyfftw.pxi'
# IF PYQG_USE_PYFFTW:
#     import pyfftw
#     pyfftw.interfaces.cache.enable()
# ELSE:
#     import jnp.fft as npfft
#     warnings.warn('No pyfftw detected. Using jax.numpy.fft')

In [7]:
class PseudoSpectralKernel:
    
    def __init__(self, nz, ny, nx, fftw_num_threads=1,
            has_q_param=0, has_uv_param=0):
        
        self.nz = nz
        self.ny = ny
        self.nx = nx
        self.nl = ny
        self.nk = nx/2 + 1
        self.a     = np.zeros((self.nz, self.nz, self.nl, self.nk), jnp.complex128)
        self.kk    = np.zeros((self.nk), jnp.float64)
        self._ik   = np.zeros((self.nk), jnp.complex128)
        self.ll    = np.zeros((self.nl), jnp.float64)
        self._il   = np.zeros((self.nl), jnp.complex128)
        self._k2l2 = np.zeros((self.nl, self.nk), jnp.float64)
        
        #### other stuff ####
        
        
        def _invert(self):
                    
            # invert qh to find ph
            self.ph = jnp.apply_over_axes(jnp.sum, jnp.multiply(self.a, self.qh), [0])
            
            # calculate spectral velocities
            self.uh = jnp.multiply(self.ph, self._il, axis=1)
            self.vh = jnp.multiply(self.ph, self._ik, axis=2)
            
            self.ifft_uh_to_u()
            self.ifft_vh_to_v()
            
            return
        
        def _do_advection(self):
        
            # multiply to get advective flux in space
            self.uq = jnp.multiply(jnp.add(self.u, self.Ubg), self.q)
            self.vq = jnp.multiply(self.v, self.q)
            
            self.fft_uq_to_uqh()
            self.fft_vq_to_vqh()
            
            # spectral divergence
            # overwrite the tendency, since the forcing gets called after
            self.dqhdt[k,j,i] = -( jnp.multiply(self.uqh, self._ik, axis=2) +
                                   jnp.multiply(self.vqh, self._il, axis=1) +
                                   jnp.multiply(self.ph, self._ikQy, axes = [0,2]) # check axes!
                                 )
            return
        
        def _do_uv_subgrid_parameterization(self):
            
            self.fft_du_to_duh()
            self.fft_dv_to_dvh()
            
            self.dqhdt = self.dqhdt +
                         ( jnp.multiply(self.dvh, self._ik, axis=2) -
                           jnp.multiply(self.duh, self._il, axis=1) )
            return
        
    